### First step

In the following step we are going to find the best joint (bid, pricing) using the implementation of Non Linear Programming optimization Algorithm provided by the Scipy Library

## Optimize with different bid

In [3]:
import numpy as np
import scipy.optimize as opt
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

from customerManager import CustomerManager
customer_path = "./config/customer_classes.json"
cManager = CustomerManager(file_path = customer_path)


# optimize
b_p = (3.0,15.0)
b_b = (1.0 , 10.0)
bnds = (b_b, b_b, b_b, b_p)

methods_with_jac = ['Nelder-Mead','Powell','CG','BFGS','Newton-CG','L-BFGS-B','TNC','COBYLA','SLSQP','trust-constr','dogleg','trust-ncg','trust-krylov','trust-exact']
methods = ['Nelder-Mead','Powell','CG','BFGS','L-BFGS-B','TNC','COBYLA','SLSQP','trust-constr']




### Over different starting points and methods

In [4]:
max_value = 0
best_method = 'none'
best_x = None

def objective( x):
    bids = x[:3]
    price = x[3]

    #Checks integrity
    if price < 3 or price > 15 : return 0.0 
    if any(b < 1 or b > 10 for b in bids): return 0.0


    res = cManager.revenue(bids,price,noise=False)

    return -sum(res)

In [5]:
bids = np.linspace(1,10,10) 
prices = np.linspace(3,15,13)

X_tmp = []
X= []
n_starting_points = 50
#generate random starting point
for b1 in bids:
    for b2 in bids:
        for b3 in bids:
            for p in prices:
                X_tmp.append([b1,b2,b3,p])

for i in range(n_starting_points):
    X.append(X_tmp[np.random.choice(len(X_tmp))])

X= np.array(X)


In [6]:
for x0 in tqdm(X):
    for m in methods:
        solution = opt.minimize(objective, x0, method = m) 
        x = solution.x

        if max_value < -objective(x):
            max_value = -objective(x)
            best_method = m
            best_x = x
print("\n")
print("Best method is " + best_method + " with a value of " + str(max_value))
print('Solution')
print('bid 1 = ' + str(best_x[0]))
print('bid 2 = ' + str(best_x[1]))
print('bid 3 = ' + str(best_x[2]))
print('price = ' + str(best_x[3]))


100%|██████████| 50/50 [00:13<00:00,  3.81it/s]

Best method is Powell with a value of 2408.710641647436
Solution
bid 1 = 3.8622487557846723
bid 2 = 2.1822222001952714
bid 3 = 2.347130672615824
price = 6.319053877879053



In [40]:

print(-objective(best_x))

2408.593195895244


### Optimize with one bid

In [35]:
max_value = 0
best_method = 'none'
best_x = None


def objective_2( x):
    bid = x[0]
    price = x[1]

    #Checks integrity
    if price < 3 or price > 15 : return 0.0 
    if bid < 1 or bid > 10: return 0.0
    bids = [bid,bid,bid]
    res = cManager.revenue(bids,price,noise=False)
    return -sum(res)

In [36]:
bids = np.linspace(1,10,10) 
prices = np.linspace(3,15,13)

X_tmp = []
X= []
n_starting_points = 50
#generate random starting point
for b in bids:
    for p in prices:
        X_tmp.append([b,p])

X= np.array(X_tmp)



In [33]:
for x0 in tqdm(X):
    for m in methods:
        solution = opt.minimize(objective_2, x0, method = m) 
        x = solution.x

        if max_value < -objective_2(x):
            max_value = -objective_2(x)
            best_method = m
            best_x = x

print("\n")
print("Best method is " + best_method + " with a value of " + str(max_value))
print('Solution')
print('bid = ' + str(best_x[0]))
print('price = ' + str(best_x[1]))

5.924219728805018]
[4.001857059308154, 4.001857059308154, 4.001857059308154]
[4.001857069308154, 4.001857069308154, 4.001857069308154]
[4.001857059308154, 4.001857059308154, 4.001857059308154]
[4.001856726171037, 4.001856726171037, 4.001856726171037]
[4.001856736171037, 4.001856736171037, 4.001856736171037]
[4.001856726171037, 4.001856726171037, 4.001856726171037]
[1.7548760991469763, 1.7548760991469763, 1.7548760991469763]
[1.7548761091469762, 1.7548761091469762, 1.7548761091469762]
[1.7548760991469763, 1.7548760991469763, 1.7548760991469763]
[1.1931308591066836, 1.1931308591066836, 1.1931308591066836]
[1.1931308691066835, 1.1931308691066835, 1.1931308691066835]
[1.1931308591066836, 1.1931308591066836, 1.1931308591066836]
[1.4740034791268295, 1.4740034791268295, 1.4740034791268295]
[1.4740034891268294, 1.4740034891268294, 1.4740034891268294]
[1.4740034791268295, 1.4740034791268295, 1.4740034791268295]
[1.6144397891369042, 1.6144397891369042, 1.6144397891369042]
[1.6144397991369042, 1.

In [34]:
print(best_x)
print(-objective_2(best_x))

[2.95498474 6.20192776]
[2.954984744564043, 2.954984744564043, 2.954984744564043]
2332.9057537722283
